# NoETL Playbook Execution Report
__Agent 007 mission report:__ _Operation completed. Martini status: shaken, not stirred._
- This notebook provides a report of the execution of a NoETL playbook using the `agent007.py` script.
- It analyzes the DuckDB database generated during the playbook run, with execution flow, step results, loop iterations, task executions, and context data captured throughout the process.
1. Run playbook with `agent007.py`.
2. Set `db_path` below to that file path.
3. Run all paragraphs to inspect tables and data.


In [1]:
%reload_ext sql


The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

## Set the database path and list all tables


In [2]:
import duckdb
import polars as pl
import os
import matplotlib.pyplot as plt
import pandas as pd
import json
import networkx as nx
from datetime import datetime

# Path to DuckDB database
db_path = '../data/noetldb/agent007.duckdb'  # Default path used by agent007.py
# Example: db_path = '/Users/kadyapam/projects/noetl/noetl/noetl/runtime/agent007.duckdb'

if not os.path.exists(db_path) and db_path != ':memory:':
    raise FileNotFoundError(f"DuckDB database not found: {db_path}")

con = duckdb.connect(db_path)

def parse_json_column(df, column_name):
    if column_name in df.columns:
        def safe_json_parse(x):
            if not x:
                return None
            try:
                return json.loads(x)
            except json.JSONDecodeError as e:
                print(f"Warning: Could not parse JSON: {e}")
                return x

        return df.with_columns([
            pl.col(column_name).map_elements(safe_json_parse, return_dtype=pl.Object)
        ])
    return df

# execution ID
latest_execution = con.execute("""
    SELECT execution_id, MAX(timestamp) as latest_time
    FROM event_log
    GROUP BY execution_id
    ORDER BY latest_time DESC
    LIMIT 1
""").fetchone()

if latest_execution:
    execution_id = latest_execution[0]
    print(f"Analyzing execution ID: {execution_id}")
else:
    print("No executions found in the database.")
    execution_id = None


Analyzing execution ID: d1b62587-7d48-47ed-85ba-cd493e9ae7fb


In [3]:
tables = con.execute("SHOW TABLES").fetchall()
for t in tables:
    print(t[0])


context
event_log
loop_state
step_result
task_result
transition
workbook
workflow


## Execution Overview

This section provides a high-level overview of the playbook execution.


In [4]:
if execution_id:
    # execution start end times
    execution_times = con.execute("""
        SELECT 
            MIN(timestamp) as start_time,
            MAX(timestamp) as end_time,
            (EXTRACT(EPOCH FROM MAX(timestamp)) - EXTRACT(EPOCH FROM MIN(timestamp))) as duration_seconds
        FROM event_log
        WHERE execution_id = ?
    """, [execution_id]).fetchone()

    # execution statistics
    execution_stats = con.execute("""
        SELECT
            (SELECT COUNT(*) FROM step_result WHERE execution_id = ?) as step_count,
            (SELECT COUNT(*) FROM task_result WHERE execution_id = ?) as task_count,
            (SELECT COUNT(*) FROM loop_state WHERE execution_id = ?) as loop_count,
            (SELECT COUNT(*) FROM event_log WHERE execution_id = ? AND event_type LIKE '%error%') as error_count
    """, [execution_id, execution_id, execution_id, execution_id]).fetchone()

    # playbook name
    playbook_info = con.execute("""
        SELECT node_name, metadata
        FROM event_log
        WHERE execution_id = ? AND event_type = 'execution_start'
        LIMIT 1
    """, [execution_id]).fetchone()

    # execution overview
    print(f"Playbook: {playbook_info[0] if playbook_info else 'Unknown'}")
    print(f"Start Time: {execution_times[0]}")
    print(f"End Time: {execution_times[1]}")
    print(f"Duration: {execution_times[2]:.2f} seconds")
    print(f"Steps Executed: {execution_stats[0]}")
    print(f"Tasks Executed: {execution_stats[1]}")
    print(f"Loops Executed: {execution_stats[2]}")
    print(f"Errors Encountered: {execution_stats[3]}")

    # playbook metadata
    if playbook_info and playbook_info[1]:
        try:
            metadata = json.loads(playbook_info[1])
            print("\nPlaybook Metadata:")
            for key, value in metadata.items():
                print(f"  {key}: {value}")
        except:
            pass
else:
    print("No execution ID available.")


Playbook: weather_iterator_example
Start Time: 2025-06-15 22:27:29.783151
End Time: 2025-06-15 22:27:32.728557
Duration: 2.95 seconds
Steps Executed: 2
Tasks Executed: 5
Loops Executed: 1
Errors Encountered: 0

Playbook Metadata:
  playbook_path: catalog/playbooks/weather_example.yaml


## Step Execution Analysis

Analyze the execution of steps in the playbook.


In [5]:
if execution_id:
    # step execution details
    step_results = pl.from_arrow(con.execute("""
        SELECT 
            step_id, 
            step_name, 
            parent_id, 
            timestamp, 
            status, 
            data, 
            error
        FROM step_result
        WHERE execution_id = ?
        ORDER BY timestamp
    """, [execution_id]).arrow())

    step_results = parse_json_column(step_results, 'data')

    # step execution summary
    step_summary = pl.from_arrow(con.execute("""
        SELECT 
            step_name, 
            COUNT(*) as execution_count,
            COUNT(CASE WHEN status = 'success' THEN 1 END) as success_count,
            COUNT(CASE WHEN status = 'error' THEN 1 END) as error_count
        FROM step_result
        WHERE execution_id = ?
        GROUP BY step_name
        ORDER BY execution_count DESC
    """, [execution_id]).arrow())

    print("Step Execution Summary:")
    display(step_summary)

    # step results
    print("\nStep Results:")
    display(step_results)
else:
    print("No execution ID available.")


Step Execution Summary:


step_name,execution_count,success_count,error_count
str,i64,i64,i64
"""start""",1,1,0
"""city_loop""",1,1,0



Step Results:


shape: (2, 7)
┌──────────────────┬───────────┬───────────┬──────────────────┬─────────┬──────────────────┬───────┐
│ step_id          ┆ step_name ┆ parent_id ┆ timestamp        ┆ status  ┆ data             ┆ error │
│ ---              ┆ ---       ┆ ---       ┆ ---              ┆ ---     ┆ ---              ┆ ---   │
│ str              ┆ str       ┆ str       ┆ datetime[μs]     ┆ str     ┆ object           ┆ str   │
╞══════════════════╪═══════════╪═══════════╪══════════════════╪═════════╪══════════════════╪═══════╡
│ a8bf342e-2731-41 ┆ start     ┆ null      ┆ 2025-06-15       ┆ success ┆ {}               ┆ null  │
│ de-a0af-3fba56…  ┆           ┆           ┆ 22:27:29.787950  ┆         ┆                  ┆       │
│ 2300d688-6c5d-44 ┆ city_loop ┆ null      ┆ 2025-06-15       ┆ success ┆ [{'fetch_and_eva ┆ null  │
│ ea-90f2-333973…  ┆           ┆           ┆ 22:27:32.034159  ┆         ┆ luate': {'data…  ┆       │
└──────────────────┴───────────┴───────────┴──────────────────┴─────────┴──────────────────┴───────┘

## Loop Execution Analysis

Analyze the execution of loops in the playbook.


In [6]:
if execution_id:
    # loop execution details
    loop_state = pl.from_arrow(con.execute("""
        SELECT 
            loop_id, 
            loop_name, 
            parent_id, 
            iterator, 
            items, 
            current_index, 
            current_item, 
            results, 
            timestamp, 
            status
        FROM loop_state
        WHERE execution_id = ?
        ORDER BY timestamp
    """, [execution_id]).arrow())

    loop_state = parse_json_column(loop_state, 'items')
    loop_state = parse_json_column(loop_state, 'current_item')
    loop_state = parse_json_column(loop_state, 'results')

    # loop execution summary
    if len(loop_state) > 0:
        print("Loop Execution Summary:")
        display(loop_state)

        # loop iteration events
        loop_events = pl.from_arrow(con.execute("""
            SELECT 
                event_id, 
                node_name, 
                event_type, 
                timestamp, 
                status, 
                duration, 
                output_result as data, 
                metadata
            FROM event_log
            WHERE execution_id = ? AND event_type LIKE '%loop%'
            ORDER BY timestamp
        """, [execution_id]).arrow())

        loop_events = parse_json_column(loop_events, 'data')
        loop_events = parse_json_column(loop_events, 'metadata')

        print("\nLoop Events:")
        display(loop_events)
    else:
        print("No loop executions found.")
else:
    print("No execution ID available.")


Loop Execution Summary:


shape: (1, 10)
┌───────────┬───────────┬───────────┬──────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ loop_id   ┆ loop_name ┆ parent_id ┆ iterator ┆ … ┆ current_i ┆ results   ┆ timestamp ┆ status    │
│ ---       ┆ ---       ┆ ---       ┆ ---      ┆   ┆ tem       ┆ ---       ┆ ---       ┆ ---       │
│ str       ┆ str       ┆ str       ┆ str      ┆   ┆ ---       ┆ object    ┆ datetime[ ┆ str       │
│           ┆           ┆           ┆          ┆   ┆ object    ┆           ┆ μs]       ┆           │
╞═══════════╪═══════════╪═══════════╪══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ a2086629- ┆ city_loop ┆ 2300d688- ┆ city     ┆ … ┆ null      ┆ [{'fetch_ ┆ 2025-06-1 ┆ completed │
│ a681-4ce4 ┆           ┆ 6c5d-44ea ┆          ┆   ┆           ┆ and_evalu ┆ 5 22:27:3 ┆           │
│ -86f0-d7c ┆           ┆ -90f2-333 ┆          ┆   ┆           ┆ ate':     ┆ 2.033245  ┆           │
│ 7f6…      ┆           ┆ 973…      ┆          ┆   ┆           ┆ {'data…   ┆           ┆           │
└───────────┴───────────┴───────────┴──────────┴───┴───────────┴───────────┴───────────┴───────────┘


Loop Events:


shape: (8, 8)
┌────────────┬────────────┬────────────┬────────────┬───────────┬──────────┬───────────┬───────────┐
│ event_id   ┆ node_name  ┆ event_type ┆ timestamp  ┆ status    ┆ duration ┆ data      ┆ metadata  │
│ ---        ┆ ---        ┆ ---        ┆ ---        ┆ ---       ┆ ---      ┆ ---       ┆ ---       │
│ str        ┆ str        ┆ str        ┆ datetime[μ ┆ str       ┆ f64      ┆ object    ┆ object    │
│            ┆            ┆            ┆ s]         ┆           ┆          ┆           ┆           │
╞════════════╪════════════╪════════════╪════════════╪═══════════╪══════════╪═══════════╪═══════════╡
│ fc22a1be-b ┆ city_loop  ┆ loop_start ┆ 2025-06-15 ┆ in_progre ┆ 0.0      ┆ null      ┆ {'item_co │
│ 5e9-4747-8 ┆            ┆            ┆ 22:27:29.7 ┆ ss        ┆          ┆           ┆ unt': 3,  │
│ 8d5-91fa5c ┆            ┆            ┆ 95949      ┆           ┆          ┆           ┆ 'iterator │
│ …          ┆            ┆            ┆            ┆           ┆          ┆           ┆ ': …      │
│ da612e3f-3 ┆ city_loop[ ┆ loop_itera ┆ 2025-06-15 ┆ in_progre ┆ 0.0      ┆ null      ┆ {'index': │
│ bf3-4e3c-9 ┆ 0]         ┆ tion       ┆ 22:27:29.7 ┆ ss        ┆          ┆           ┆ 0,        │
│ ac7-3d46f5 ┆            ┆            ┆ 98667      ┆           ┆          ┆           ┆ 'item':   │
│ …          ┆            ┆            ┆            ┆           ┆          ┆           ┆ {'name':  │
│            ┆            ┆            ┆            ┆           ┆          ┆           ┆ …         │
│ ca0461c7-d ┆ city_loop[ ┆ loop_itera ┆ 2025-06-15 ┆ success   ┆ 0.0      ┆ {'fetch_a ┆ {'index': │
│ de4-4525-9 ┆ 0]         ┆ tion_compl ┆ 22:27:30.6 ┆           ┆          ┆ nd_evalua ┆ 0,        │
│ 8a1-394aeb ┆            ┆ ete        ┆ 19030      ┆           ┆          ┆ te':      ┆ 'item':   │
│ …          ┆            ┆            ┆            ┆           ┆          ┆ {'data'…  ┆ {'name':  │
│            ┆            ┆            ┆            ┆           ┆          ┆           ┆ …         │
│ ca0e3adf-3 ┆ city_loop[ ┆ loop_itera ┆ 2025-06-15 ┆ in_progre ┆ 0.0      ┆ null      ┆ {'index': │
│ 4c6-4404-8 ┆ 1]         ┆ tion       ┆ 22:27:30.6 ┆ ss        ┆          ┆           ┆ 1,        │
│ 9ee-2be953 ┆            ┆            ┆ 20997      ┆           ┆          ┆           ┆ 'item':   │
│ …          ┆            ┆            ┆            ┆           ┆          ┆           ┆ {'name':  │
│            ┆            ┆            ┆            ┆           ┆          ┆           ┆ …         │
│ 353f7197-d ┆ city_loop[ ┆ loop_itera ┆ 2025-06-15 ┆ success   ┆ 0.0      ┆ {'fetch_a ┆ {'index': │
│ 41c-4e56-8 ┆ 1]         ┆ tion_compl ┆ 22:27:31.3 ┆           ┆          ┆ nd_evalua ┆ 1,        │
│ 44c-142ef6 ┆            ┆ ete        ┆ 34112      ┆           ┆          ┆ te':      ┆ 'item':   │
│ …          ┆            ┆            ┆            ┆           ┆          ┆ {'data'…  ┆ {'name':  │
│            ┆            ┆            ┆            ┆           ┆          ┆           ┆ …         │
│ 8615fbd3-b ┆ city_loop[ ┆ loop_itera ┆ 2025-06-15 ┆ in_progre ┆ 0.0      ┆ null      ┆ {'index': │
│ f89-47c5-a ┆ 2]         ┆ tion       ┆ 22:27:31.3 ┆ ss        ┆          ┆           ┆ 2,        │
│ 781-657271 ┆            ┆            ┆ 36123      ┆           ┆          ┆           ┆ 'item':   │
│ …          ┆            ┆            ┆            ┆           ┆          ┆           ┆ {'name':  │
│            ┆            ┆            ┆            ┆           ┆          ┆           ┆ …         │
│ 8a4f122d-d ┆ city_loop[ ┆ loop_itera ┆ 2025-06-15 ┆ success   ┆ 0.0      ┆ {'fetch_a ┆ {'index': │
│ 02c-4959-a ┆ 2]         ┆ tion_compl ┆ 22:27:32.0 ┆           ┆          ┆ nd_evalua ┆ 2,        │
│ 4da-f833cb ┆            ┆ ete        ┆ 32587      ┆           ┆          ┆ te':      ┆ 'item':   │
│ …          ┆            ┆            ┆            ┆           ┆          ┆ {'data'…  ┆ {'name':  │
│            ┆            ┆            ┆       

## Task Execution Analysis

Analyze the execution of tasks in the playbook.


In [7]:
if execution_id:
    # task execution details
    task_results = pl.from_arrow(con.execute("""
        SELECT 
            task_id, 
            task_name, 
            task_type, 
            parent_id, 
            timestamp, 
            status, 
            data, 
            error
        FROM task_result
        WHERE execution_id = ?
        ORDER BY timestamp
    """, [execution_id]).arrow())

    task_results = parse_json_column(task_results, 'data')

    # task execution summary
    task_summary = pl.from_arrow(con.execute("""
        SELECT 
            task_name, 
            task_type,
            COUNT(*) as execution_count,
            COUNT(CASE WHEN status = 'success' THEN 1 END) as success_count,
            COUNT(CASE WHEN status = 'error' THEN 1 END) as error_count
        FROM task_result
        WHERE execution_id = ?
        GROUP BY task_name, task_type
        ORDER BY execution_count DESC
    """, [execution_id]).arrow())

    print("Task Execution Summary:")
    display(task_summary)

    # task results
    print("\nTask Results:")
    display(task_results)
else:
    print("No execution ID available.")


Task Execution Summary:


task_name,task_type,execution_count,success_count,error_count
str,str,i64,i64,i64
"""get_forecast""","""http""",4,4,0
"""evaluate_weather""","""python""",1,1,0



Task Results:


task_id,task_name,task_type,parent_id,timestamp,status,data,error
str,str,str,str,datetime[μs],str,object,str
"""40830961-bd4c-453e-b14a-7a1b9a…","""get_forecast""","""http""","""cc81ad95-7fd4-42a4-a833-a9b51c…",2025-06-15 22:27:30.608438,"""success""","{'data': {'latitude': 51.5, 'longitude': -0.120000124, 'generationtime_ms': 0.019073486328125, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 29.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C'}, 'hourly': {'time': ['2025-06-16T00:00', '2025-06-16T01:00', '2025-06-16T02:00', '2025-06-16T03:00', '2025-06-16T04:00', '2025-06-16T05:00', '2025-06-16T06:00', '2025-06-16T07:00', '2025-06-16T08:00', '2025-06-16T09:00', '2025-06-16T10:00', '2025-06-16T11:00', '2025-06-16T12:00', '2025-06-16T13:00', '2025-06-16T14:00', '2025-06-16T15:00', '2025-06-16T16:00', '2025-06-16T17:00', '2025-06-16T18:00', '2025-06-16T19:00', '2025-06-16T20:00', '2025-06-16T21:00', '2025-06-16T22:00', '2025-06-16T23:00'], 'temperature_2m': [17.1, 16.5, 16.1, 15.5, 15.1, 14.9, 15.4, 16.5, 17.9, 19.4, 20.6, 21.8, 22.8, 23.6, 24.8, 25.1, 25.5, 25.8, 25.5, 23.7, 22.1, 21.2, 20.2, 19.4]}}, 'status_code': 200, 'headers': {'date': 'Mon, 16 Jun 2025 05:27:30 GMT', 'content-type': 'application/json; charset=utf-8', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'content-encoding': 'deflate'}}",null
"""4e69dfff-b9a5-4de0-b47e-537046…","""get_forecast""","""http""","""5399339c-bd8e-4da0-914d-cd823a…",2025-06-15 22:27:31.324762,"""success""","{'data': {'latitude': 48.84, 'longitude': 2.3599997, 'generationtime_ms': 0.017762184143066406, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 46.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C'}, 'hourly': {'time': ['2025-06-16T00:00', '2025-06-16T01:00', '2025-06-16T02:00', '2025-06-16T03:00', '2025-06-16T04:00', '2025-06-16T05:00', '2025-06-16T06:00', '2025-06-16T07:00', '2025-06-16T08:00', '2025-06-16T09:00', '2025-06-16T10:00', '2025-06-16T11:00', '2025-06-16T12:00', '2025-06-16T13:00', '2025-06-16T14:00', '2025-06-16T15:00', '2025-06-16T16:00', '2025-06-16T17:00', '2025-06-16T18:00', '2025-06-16T19:00', '2025-06-16T20:00', '2025-06-16T21:00', '2025-06-16T22:00', '2025-06-16T23:00'], 'temperature_2m': [16.3, 15.7, 15.2, 14.5, 14.1, 14.1, 14.6, 15.6, 17.0, 18.7, 20.5, 22.0, 23.3, 24.1, 25.1, 25.4, 26.0, 25.5, 25.2, 24.4, 23.7, 22.8, 21.9, 21.3]}}, 'status_code': 200, 'headers': {'date': 'Mon, 16 Jun 2025 05:27:31 GMT', 'content-type': 'application/json; charset=utf-8', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'content-encoding': 'deflate'}}",null
"""b5470898-5849-4334-b3bf-33b4d7…","""get_forecast""","""http""","""be9012c0-0d82-4198-b607-dd8a43…",2025-06-15 22:27:32.024349,"""success""","{'data': {'latitude': 52.52, 'longitude': 13.419998, 'generationtime_ms': 0.030159950256347656, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 38.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C'}, 'hourly': {'time': ['2025-06-16T00:00', '2025-06-16T01:00', '2025-06-16T02:00', '2025-06-16T03:00', '2025-06-16T04:00', '2025-06-16T05:00', '2025-06-16T06:00', '2025-06-16T07:00', '2025-06-16T08:00', '2025-06-16T09:00', '2025-06-16T10:00', '2025-06-16T11:00', '2025-06-16T12:00', '2025-06-16T13:00', '2025-06-16T14:00', '2025-06-16T15:00', '2025-06-16T16:00', '2025-06-16T17:00', '2025-06-16T18:00', '2025-06-16T19:00', '2025-06-16T20:00', '2025-06-16T21:00', '2025-06-16T22:00', '2025-06-16T23:00'], 'temperature_2m': [16.9, 16.9, 16.8, 16.9, 16.7, 16.5, 15.6, 17.0, 18.3, 19.5, 19.3, 18.8, 19.7, 20.5, 21.3, 22.2, 22.7, 22.5, 22.1, 21.2, 20.1, 18.9, 17.9, 17.0]}}, 'status_code': 200, 'headers': {'date': 'Mon, 16 Jun 2025 05:27:31 GMT', 'content-type': 'application/json; charset=utf-8', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'content-encoding': 'deflate'}}",null
"""090afa1a-f6ce-4d30-b47b-eb39f9…",

## Context Analysis

Analyze the context data during playbook execution.


In [8]:
if execution_id:
    # context data
    context_data = pl.from_arrow(con.execute("""
        SELECT 
            key, 
            value, 
            timestamp
        FROM context
        WHERE execution_id = ?
        ORDER BY timestamp
    """, [execution_id]).arrow())

    context_data = parse_json_column(context_data, 'value')

    if len(context_data) > 0:
        print("Context Data:")
        display(context_data)

        # context state
        final_context = {}
        for _, row in context_data.to_pandas().iterrows():
            final_context[row['key']] = row.get('value')

        print("\nFinal Context State:")
        for key, value in final_context.items():
            print(f"\n{key}:")
            if isinstance(value, (dict, list)):
                print(json.dumps(value, indent=2))
            else:
                print(value)
    else:
        print("No context data found.")
else:
    print("No execution ID found.")


Context Data:


shape: (10, 3)
┌───────────────────────┬─────────────────────────────────┬────────────────────────────┐
│ key                   ┆ value                           ┆ timestamp                  │
│ ---                   ┆ ---                             ┆ ---                        │
│ str                   ┆ object                          ┆ datetime[μs]               │
╞═══════════════════════╪═════════════════════════════════╪════════════════════════════╡
│ workload              ┆ {'jobId': '{{ job.uuid }}', 's… ┆ 2025-06-15 22:27:29.781039 │
│ execution_start       ┆ 2025-06-15T22:27:29.782784      ┆ 2025-06-15 22:27:29.782794 │
│ city_loop_results     ┆ [{'fetch_and_evaluate': {'data… ┆ 2025-06-15 22:27:32.033757 │
│ base_url              ┆ https://api.open-meteo.com/v1   ┆ 2025-06-15 22:27:32.038979 │
│ get_forecast          ┆ {'result': {'data': {'latitude… ┆ 2025-06-15 22:27:32.716597 │
│ city                  ┆ {'name': 'Berlin', 'lat': 52.5… ┆ 2025-06-15 22:27:32.719858 │
│ forecast_data         ┆ {'latitude': 52.52, 'longitude… ┆ 2025-06-15 22:27:32.720229 │
│ temperature_threshold ┆ 25                              ┆ 2025-06-15 22:27:32.720557 │
│ evaluate_weather      ┆ {'result': {'city': 'Berlin', … ┆ 2025-06-15 22:27:32.726590 │
│ result                ┆ {'city': 'Berlin', 'max_temp':… ┆ 2025-06-15 22:27:32.726876 │
└───────────────────────┴─────────────────────────────────┴────────────────────────────┘


Final Context State:

workload:
{
  "jobId": "{{ job.uuid }}",
  "state": "ready",
  "cities": [
    {
      "name": "London",
      "lat": 51.51,
      "lon": -0.13
    },
    {
      "name": "Paris",
      "lat": 48.85,
      "lon": 2.35
    },
    {
      "name": "Berlin",
      "lat": 52.52,
      "lon": 13.41
    }
  ],
  "base_url": "https://api.open-meteo.com/v1",
  "temperature_threshold": 25
}

execution_start:
2025-06-15T22:27:29.782784

city_loop_results:
[
  {
    "fetch_and_evaluate": {
      "data": {
        "latitude": 51.5,
        "longitude": -0.120000124,
        "generationtime_ms": 0.019073486328125,
        "utc_offset_seconds": 0,
        "timezone": "GMT",
        "timezone_abbreviation": "GMT",
        "elevation": 29.0,
        "hourly_units": {
          "time": "iso8601",
          "temperature_2m": "\u00b0C"
        },
        "hourly": {
          "time": [
            "2025-06-16T00:00",
            "2025-06-16T01:00",
            "2025-06-16T02:00",
  

## Preview `event_log` table


In [9]:
table_name = 'event_log'
event_log_df = pl.from_arrow(con.execute(f"SELECT * FROM {table_name} LIMIT 20").arrow())
event_log_df


execution_id,event_id,parent_event_id,timestamp,event_type,node_id,node_name,node_type,status,duration,input_context,output_result,metadata
str,str,str,datetime[μs],str,str,str,str,str,f64,str,str,str
"""a497e03e-1e8a-4708-97da-cd981e…","""11d58c32-68b0-45e6-818a-295d7a…",null,2025-06-15 21:06:41.543364,"""execution_start""","""a497e03e-1e8a-4708-97da-cd981e…","""weather_iterator_example""","""playbook""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""playbook_path"": ""catalog/pla…"
"""a497e03e-1e8a-4708-97da-cd981e…","""0992c224-020a-40d0-ad9d-836927…",null,2025-06-15 21:06:41.544493,"""step_start""","""09485942-784e-4a03-987b-26017f…","""start""","""step""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""step_type"": ""standard""}"""
"""a497e03e-1e8a-4708-97da-cd981e…","""bf7026af-a965-496e-99cf-eda7f0…","""0992c224-020a-40d0-ad9d-836927…",2025-06-15 21:06:41.545576,"""step_complete""","""09485942-784e-4a03-987b-26017f…","""start""","""step""","""success""",0.001186,"""{""jobId"": ""{{ job.uuid }}"", ""s…","""{}""","""{""step_type"": ""standard""}"""
"""a497e03e-1e8a-4708-97da-cd981e…","""6193ea67-8155-4e2b-b6be-4de1b3…","""11d58c32-68b0-45e6-818a-295d7a…",2025-06-15 21:06:41.549975,"""step_transition""","""a497e03e-1e8a-4708-97da-cd981e…","""transition_to_city_loop""","""transition""","""success""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""from_step"": ""start"", ""to_ste…"
"""a497e03e-1e8a-4708-97da-cd981e…","""0991095f-57c4-45a8-b17f-c0c999…",null,2025-06-15 21:06:41.551111,"""step_start""","""3f85e793-7aa8-4823-b0c2-7c7533…","""city_loop""","""step""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""step_type"": ""standard""}"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""a497e03e-1e8a-4708-97da-cd981e…","""f6bfa8c6-a980-48d2-b7af-13205d…",null,2025-06-15 21:06:41.566602,"""step_start""","""575c80c1-a509-4420-b63d-b432e3…","""fetch_and_evaluate""","""step""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""step_type"": ""standard""}"""
"""a497e03e-1e8a-4708-97da-cd981e…","""9ca98b4b-81e4-4eab-9e35-b5eb95…","""f6bfa8c6-a980-48d2-b7af-13205d…",2025-06-15 21:06:41.567217,"""task_execute""","""b1764051-4ec0-45f0-bb10-fdc9f6…","""get_forecast""","""task.http""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""task_type"": ""http""}"""
"""a497e03e-1e8a-4708-97da-cd981e…","""9b6dba54-6c82-4610-879f-b9d031…","""9ca98b4b-81e4-4eab-9e35-b5eb95…",2025-06-15 21:06:41.568354,"""task_start""","""db3feb39-0b0c-4961-b35d-9ce76a…","""get_forecast""","""http""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""method"": ""GET"", ""endpoint"": …"


## Steps resluts


In [10]:
pl.from_arrow(con.execute("SELECT * FROM step_result ORDER BY timestamp DESC LIMIT 10").arrow())


execution_id,step_id,step_name,parent_id,timestamp,status,data,error
str,str,str,str,datetime[μs],str,str,str
"""d1b62587-7d48-47ed-85ba-cd493e…","""2300d688-6c5d-44ea-90f2-333973…","""city_loop""",null,2025-06-15 22:27:32.034159,"""success""","""[{""fetch_and_evaluate"": {""data…",null
"""d1b62587-7d48-47ed-85ba-cd493e…","""a8bf342e-2731-41de-a0af-3fba56…","""start""",null,2025-06-15 22:27:29.787950,"""success""","""{}""",null
"""a893a719-bd35-412a-a4cb-004505…","""66069bff-c726-44a9-96bb-56c98b…","""city_loop""",null,2025-06-15 22:26:26.327437,"""success""","""[{""fetch_and_evaluate"": {""data…",null
"""a893a719-bd35-412a-a4cb-004505…","""4ea67d50-32b5-49ee-8001-f322b8…","""start""",null,2025-06-15 22:26:24.021870,"""success""","""{}""",null
"""a1135fff-b063-4d81-89a3-e18c2e…","""7fef4542-413e-4481-969d-6b0e61…","""city_loop""",null,2025-06-15 22:24:09.627491,"""success""","""[{""fetch_and_evaluate"": {""data…",null
"""a1135fff-b063-4d81-89a3-e18c2e…","""0d2b6da4-cdb0-476e-9944-51cfe9…","""start""",null,2025-06-15 22:24:07.409725,"""success""","""{}""",null
"""8ae04ae6-ce97-4159-9e0a-c361fc…","""cb1f4f5c-bb4d-40da-9902-43a063…","""city_loop""",null,2025-06-15 22:16:35.012856,"""success""","""[{""fetch_and_evaluate"": {""data…",null
"""8ae04ae6-ce97-4159-9e0a-c361fc…","""395845e9-db9a-423f-adad-85811a…","""start""",null,2025-06-15 22:16:32.800689,"""success""","""{}""",null
"""d0c3efa7-d92a-40e4-91d9-6a448a…","""2e55af5e-4a62-4dd4-bd3c-31ccd1…","""city_loop""",null,2025-06-15 21:53:50.235179,"""success""","""[{""fetch_and_evaluate"": {""data…",null


## Task results

In [11]:
con.sql("SELECT * FROM task_result ORDER BY timestamp DESC LIMIT 10").show()

┌──────────────────────────────────────┬──────────────────────────────────────┬──────────────────┬───────────┬──────────────────────────────────────┬────────────────────────────┬─────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [12]:
%sql con --alias duckdb
%sql SELECT * FROM task_result ORDER BY timestamp DESC LIMIT 10

Running query in 'duckdb'

execution_id,task_id,task_name,task_type,parent_id,timestamp,status,data,error
d1b62587-7d48-47ed-85ba-cd493e9ae7fb,5f393658-6161-4c08-aba3-d8d24c4991d2,evaluate_weather,python,6a683417-0e54-48e0-9515-0bb0dffaf47a,2025-06-15 22:27:32.724332,success,"{""city"": ""Berlin"", ""max_temp"": 22.7, ""alert"": false}",None
d1b62587-7d48-47ed-85ba-cd493e9ae7fb,090afa1a-f6ce-4d30-b47b-eb39f9e60e8a,get_forecast,http,44c9ebd2-920c-43f8-8da1-7a068b8610b6,2025-06-15 22:27:32.711515,success,"{""data"": {""latitude"": 52.52, ""longitude"": 13.419998, ""generationtime_ms"": 0.024318695068359375, ""utc_offset_seconds"": 0, ""timezone"": ""GMT"", ""timezone_abbreviation"": ""GMT"", ""elevation"": 38.0, ""hourly_units"": {""time"": ""iso8601"", ""temperature_2m"": ""\u00b0C""}, ""hourly"": {""time"": [""2025-06-16T00:00"", ""2025-06-16T01:00"", ""2025-06-16T02:00"", ""2025-06-16T03:00"", ""2025-06-16T04:00"", ""2025-06-16T05:00"", ""2025-06-16T06:00"", ""2025-06-16T07:00"", ""2025-06-16T08:00"", ""2025-06-16T09:00"", ""2025-06-16T10:00"", ""2025-06-16T11:00"", ""2025-06-16T12:00"", ""2025-06-16T13:00"", ""2025-06-16T14:00"", ""2025-06-16T15:00"", ""2025-06-16T16:00"", ""2025-06-16T17:00"", ""2025-06-16T18:00"", ""2025-06-16T19:00"", ""2025-06-16T20:00"", ""2025-06-16T21:00"", ""2025-06-16T22:00"", ""2025-06-16T23:00""], ""temperature_2m"": [16.9, 16.9, 16.8, 16.9, 16.7, 16.5, 15.6, 17.0, 18.3, 19.5, 19.3, 18.8, 19.7, 20.5, 21.3, 22.2, 22.7, 22.5, 22.1, 21.2, 20.1, 18.9, 17.9, 17.0]}}, ""status_code"": 200, ""headers"": {""date"": ""Mon, 16 Jun 2025 05:27:32 GMT"", ""content-type"": ""application/json; charset=utf-8"", ""transfer-encoding"": ""chunked"", ""connection"": ""keep-alive"", ""content-encoding"": ""deflate""}}",None
d1b62587-7d48-47ed-85ba-cd493e9ae7fb,b5470898-5849-4334-b3bf-33b4d762118a,get_forecast,http,be9012c0-0d82-4198-b607-dd8a4322a6ac,2025-06-15 22:27:32.024349,success,"{""data"": {""latitude"": 52.52, ""longitude"": 13.419998, ""generationtime_ms"": 0.030159950256347656, ""utc_offset_seconds"": 0, ""timezone"": ""GMT"", ""timezone_abbreviation"": ""GMT"", ""elevation"": 38.0, ""hourly_units"": {""time"": ""iso8601"", ""temperature_2m"": ""\u00b0C""}, ""hourly"": {""time"": [""2025-06-16T00:00"", ""2025-06-16T01:00"", ""2025-06-16T02:00"", ""2025-06-16T03:00"", ""2025-06-16T04:00"", ""2025-06-16T05:00"", ""2025-06-16T06:00"", ""2025-06-16T07:00"", ""2025-06-16T08:00"", ""2025-06-16T09:00"", ""2025-06-16T10:00"", ""2025-06-16T11:00"", ""2025-06-16T12:00"", ""2025-06-16T13:00"", ""2025-06-16T14:00"", ""2025-06-16T15:00"", ""2025-06-16T16:00"", ""2025-06-16T17:00"", ""2025-06-16T18:00"", ""2025-06-16T19:00"", ""2025-06-16T20:00"", ""2025-06-16T21:00"", ""2025-06-16T22:00"", ""2025-06-16T23:00""], ""temperature_2m"": [16.9, 16.9, 16.8, 16.9, 16.7, 16.5, 15.6, 17.0, 18.3, 19.5, 19.3, 18.8, 19.7, 20.5, 21.3, 22.2, 22.7, 22.5, 22.1, 21.2, 20.1, 18.9, 17.9, 17.0]}}, ""status_code"": 200, ""headers"": {""date"": ""Mon, 16 Jun 2025 05:27:31 GMT"", ""content-type"": ""application/json; charset=utf-8"", ""transfer-encoding"": ""chunked"", ""connection"": ""keep-alive"", ""content-encoding"": ""deflate""}}",None
d1b62587-7d48-47ed-85ba-cd493e9ae7fb,4e69dfff-b9a5-4de0-b47e-5370462a71ef,get_forecast,http,5399339c-bd8e-4da0-914d-cd823a591814,2025-06-15 22:27:31.324762,success,"{""data"": {""latitude"": 48.84, ""longitude"": 2.3599997, ""generationtime_ms"": 0.017762184143066406, ""utc_offset_seconds"": 0, ""timezone"": ""GMT"", ""timezone_abbreviation"": ""GMT"", ""elevation"": 46.0, ""hourly_units"": {""time"": ""iso8601"", ""temperature_2m"": ""\u00b0C""}, ""hourly"": {""time"": [""2025-06-16T00:00"", ""2025-06-16T01:00"", ""2025-06-16T02:00"", ""2025-06-16T03:00"", ""2025-06-16T04:00"", ""2025-06-16T05:00"", ""2025-06-16T06:00"", ""2025-06-16T07:00"", ""2025-06-16T08:00"", ""2025-06-16T09:00"", ""2025-06-16T10:00"", ""2025-06-16T11:00"", ""2025-06-16T12:00"", ""2025-06-16T13:00"", ""2025-06-16T14:00"", ""2025-06-16T15:00"", ""2025-06-16T16:0